In [1]:
# --------------------------------------
import warnings

warnings.filterwarnings("ignore")

# --------------------------------------
import ibis

ibis.options.interactive = True

# --------------------------------------
import humanize

# --------------------------------------
from streetscapes import conf
from streetscapes import info
from streetscapes import logger
from streetscapes import functions as scf

# Converting CSV files to Parquet and merging them together

The CSV files of the original Global Streetscapes dataset add up to 64GB in total. Moreover, data is split in several files, which can make it a bit cumbersome to work with. Here, we convert the data to Parquet, which reduces file size and makes it easier to load and manipulate the data. 

Additionally, we combine columns from several sources into a single dataset that should serve most usecases.

First, let's declare some storage locations.

In [2]:
CSV_DIR = scf.mkdir(conf.DATA_DIR / "csv")
PARQUET_DIR = scf.mkdir(conf.DATA_DIR / "parquet")
MERGED_DIR = scf.mkdir(conf.DATA_DIR / "merged")

Create a DuckDB connection via Ibis. This will be used to manipulate all the data below.

In [3]:
con = ibis.connect("duckdb://")

Show some metadata about the available CSV files.

In [4]:
info.render_info_csv()

- climate.csv - Contains the Koppen climate zone associated with each image's location.
  The calculation is as accurate as the location of the image given by the source, which also relies on the accuracy of the capturing devices. The accuracy could also be affected by the accuracy of the Koppen climate zone classification API from https://github.com/sco-tt/Climate-Zone-API.
    - uuid (string) - Universally Unique IDentifier, unique for every image
    - source (string) - Source of the image, either Mapillary or KartaView
    - orig_id (int) - Original ID of the image as specified by Mapillary or KartaView
    - koppen_geiger_zone (string) - A zone code to identify the Koppen climate zone
    - zone_description (string) - Short description of the climate zone
- contextual.csv - Contains the eight contextual attributes inferred for each image.
  Please refer to Table 3 in the paper for information on accuracy.
    - uuid (string) - Universally Unique IDentifier, unique for every image


We will select and download a subset of the available CSV files to work with below.

In [5]:
file_names = [
    "simplemaps",
    "perception",
    "osm",
    "places365",
    "segmentation",
    "contextual",
    "metadata_common_attributes",
    "ghsl",
]

scf.download_files_hf([f"{f}.csv" for f in file_names], local_dir=CSV_DIR)

Streetscapes | 2025-01-30@15:16:15 | Downloading files from HuggingFace Hub...


simplemaps.csv:   0%|          | 0.00/1.60G [00:00<?, ?B/s]

perception.csv:   0%|          | 0.00/940M [00:00<?, ?B/s]

osm.csv:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

places365.csv:   0%|          | 0.00/733M [00:00<?, ?B/s]

segmentation.csv:   0%|          | 0.00/4.32G [00:00<?, ?B/s]

contextual.csv:   0%|          | 0.00/1.16G [00:00<?, ?B/s]

metadata_common_attributes.csv:   0%|          | 0.00/2.38G [00:00<?, ?B/s]

ghsl.csv:   0%|          | 0.00/798M [00:00<?, ?B/s]

In [6]:
# List of CSV file paths
csv_files = list(CSV_DIR.glob("*.csv"))

# Convert all csvs in data dir to parquet
for file_name in csv_files:

    # Compile the Parquet file name.
    parquet_file = PARQUET_DIR / file_name.with_suffix('.parquet').name

    if parquet_file.exists():
        logger.info(f"File '{parquet_file}' exists, skipping.")
        continue

    logger.info(f"Converting '{file_name.name}' into '{parquet_file.name}'")
    con.read_csv(file_name).to_parquet(parquet_file, compression="ZSTD")

logger.info("Done!")

# List of Parquet file paths
parquet_files = list(PARQUET_DIR.glob("*.parquet"))

Streetscapes | 2025-01-30@15:34:49 | Converting 'metadata_common_attributes.csv' into 'metadata_common_attributes.parquet'


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Streetscapes | 2025-01-30@15:35:01 | Converting 'ghsl.csv' into 'ghsl.parquet'
Streetscapes | 2025-01-30@15:35:04 | Converting 'places365.csv' into 'places365.parquet'


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Streetscapes | 2025-01-30@15:35:06 | Converting 'contextual.csv' into 'contextual.parquet'


...

Streetscapes | 2025-01-30@15:35:40 | Converting 'osm.csv' into 'osm.parquet'


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Streetscapes | 2025-01-30@15:35:51 | Converting 'perception.csv' into 'perception.parquet'


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Streetscapes | 2025-01-30@15:35:56 | Done!


Verify that the CSV and Parquet files contain the same information

In [7]:
csv_file = con.read_csv(CSV_DIR / "osm.csv")
csv_file.head()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┓
┃ uuid                                 ┃ source    ┃ orig_id          ┃ snap_dist ┃ u            ┃ v            ┃ key     ┃ osmid                              ┃ oneway  ┃ lanes  ┃ name                              ┃ highway   ┃ type_highway ┃ maxspeed ┃ junction ┃ length  ┃ from         ┃ to           ┃ ref    ┃ tunnel ┃ bridge ┃ service ┃ access ┃ road_width ┃ area    ┃ est_width ┃ reversed ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━┩
│ string                               │ string    │ int64            │ float64   │ float64      │ float64      │ float64 │ string                             │ boolean │ string │ string                            │ string    │ string       │ string   │ string   │ float64 │ float64      │ float64      │ string │ string │ string │ string  │ string │ string     │ boolean │ string    │ string   │
├──────────────────────────────────────┼───────────┼──────────────────┼───────────┼──────────────┼──────────────┼─────────┼────────────────────────────────────┼─────────┼────────┼───────────────────────────────────┼───────────┼──────────────┼──────────┼──────────┼─────────┼──────────────┼──────────────┼────────┼────────┼────────┼─────────┼────────┼────────────┼─────────┼───────────┼──────────┤
│ bc5862a5-5e4c-4f74-bdd5-598e140dbb8f │ Mapillary │  941074783267368 │  7.616701 │ 2.694255e+08 │ 2.694255e+08 │     0.0 │ 1017964492                         │ True    │ 2      │ Carrera 33                        │ tertiary  │ drive        │ NULL     │ NULL     │  62.864 │ 2.694255e+08 │ 2.694255e+08 │ NULL   │ NULL   │ NULL   │ NULL    │ NULL   │ NULL       │ NULL    │ NULL      │ False    │
│ 4d445c9a-03e2-4dda-a494-caf907ad1620 │ Mapillary │ 2822175224761380 │  4.063028 │ 1.605031e+08 │ 9.945572e+09 │     0.0 │ 1085398478                         │ False   │ 1      │ Burgstraße                        │ footway   │ walk         │ 30       │ NULL     │ 118.549 │ 9.945572e+09 │ 1.605031e+08 │ NULL   │ NULL   │ NULL   │ NULL    │ NULL   │ 12.2       │ NULL    │ NULL      │ False    │
│ 57713c58-62b2-465b-9df3-087b6d970603 │ Mapillary │  387731282398462 │  2.806557 │ 1.825447e+09 │ 5.656773e+09 │     0.0 │ 171530233                          │ False   │ NULL   │ 15th Street Northwest Cycle Track │ footway   │ walk         │ NULL     │ NULL     │  53.521 │ 5.656773e+09 │ 1.825447e+09 │ NULL   │ NULL   │ NULL   │ NULL    │ NULL   │ NULL       │ NULL    │ NULL      │ False    │
│ 5ce677fe-1f66-4a6e-a162-88ff5d6cd80a │ Mapillary │ 4331166880261832 │  5.224566 │ 2.682150e+09 │ 7.694047e+09 │     0.0 │ [771744361, 1108989117, 771743389] │ True    │ 3      │ Jalan Pudu                        │ secondary │ drive        │ NULL     │ NULL     │ 110.317 │ 7.694047e+09 │ 2.682150e+09 │ NULL   │ NULL   │ NULL   │ NULL    │ NULL   │ NULL       │ NULL    │ NULL      │ False    │
│ 54fb768e-2864-4ed0-a658-e72f7a66cbc0 │ Mapillary │  808249360075988 │  5.111907 │ 2.473251e+08 │ 1.783876e+09 │     0.0 │ 71197419                           │ True    │ 1      │ Avenue de la Liberté              │ primary   │ drive        │ 50       │ NULL     │ 106.098 │ 2.473251e+08 │ 1.783876e+09 │ N 3    │ NULL   │ NULL   │ NULL    │ NULL   │ NULL       │ NULL    │ NULL      │ False    │
└─────────────────────────────

In [8]:
parquet_file = con.read_parquet(PARQUET_DIR / "osm.parquet")
parquet_file.head()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┓
┃ uuid                                 ┃ source    ┃ orig_id          ┃ snap_dist ┃ u            ┃ v            ┃ key     ┃ osmid                              ┃ oneway  ┃ lanes  ┃ name                              ┃ highway   ┃ type_highway ┃ maxspeed ┃ junction ┃ length  ┃ from         ┃ to           ┃ ref    ┃ tunnel ┃ bridge ┃ service ┃ access ┃ road_width ┃ area    ┃ est_width ┃ reversed ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━┩
│ string                               │ string    │ int64            │ float64   │ float64      │ float64      │ float64 │ string                             │ boolean │ string │ string                            │ string    │ string       │ string   │ string   │ float64 │ float64      │ float64      │ string │ string │ string │ string  │ string │ string     │ boolean │ string    │ string   │
├──────────────────────────────────────┼───────────┼──────────────────┼───────────┼──────────────┼──────────────┼─────────┼────────────────────────────────────┼─────────┼────────┼───────────────────────────────────┼───────────┼──────────────┼──────────┼──────────┼─────────┼──────────────┼──────────────┼────────┼────────┼────────┼─────────┼────────┼────────────┼─────────┼───────────┼──────────┤
│ bc5862a5-5e4c-4f74-bdd5-598e140dbb8f │ Mapillary │  941074783267368 │  7.616701 │ 2.694255e+08 │ 2.694255e+08 │     0.0 │ 1017964492                         │ True    │ 2      │ Carrera 33                        │ tertiary  │ drive        │ NULL     │ NULL     │  62.864 │ 2.694255e+08 │ 2.694255e+08 │ NULL   │ NULL   │ NULL   │ NULL    │ NULL   │ NULL       │ NULL    │ NULL      │ False    │
│ 4d445c9a-03e2-4dda-a494-caf907ad1620 │ Mapillary │ 2822175224761380 │  4.063028 │ 1.605031e+08 │ 9.945572e+09 │     0.0 │ 1085398478                         │ False   │ 1      │ Burgstraße                        │ footway   │ walk         │ 30       │ NULL     │ 118.549 │ 9.945572e+09 │ 1.605031e+08 │ NULL   │ NULL   │ NULL   │ NULL    │ NULL   │ 12.2       │ NULL    │ NULL      │ False    │
│ 57713c58-62b2-465b-9df3-087b6d970603 │ Mapillary │  387731282398462 │  2.806557 │ 1.825447e+09 │ 5.656773e+09 │     0.0 │ 171530233                          │ False   │ NULL   │ 15th Street Northwest Cycle Track │ footway   │ walk         │ NULL     │ NULL     │  53.521 │ 5.656773e+09 │ 1.825447e+09 │ NULL   │ NULL   │ NULL   │ NULL    │ NULL   │ NULL       │ NULL    │ NULL      │ False    │
│ 5ce677fe-1f66-4a6e-a162-88ff5d6cd80a │ Mapillary │ 4331166880261832 │  5.224566 │ 2.682150e+09 │ 7.694047e+09 │     0.0 │ [771744361, 1108989117, 771743389] │ True    │ 3      │ Jalan Pudu                        │ secondary │ drive        │ NULL     │ NULL     │ 110.317 │ 7.694047e+09 │ 2.682150e+09 │ NULL   │ NULL   │ NULL   │ NULL    │ NULL   │ NULL       │ NULL    │ NULL      │ False    │
│ 54fb768e-2864-4ed0-a658-e72f7a66cbc0 │ Mapillary │  808249360075988 │  5.111907 │ 2.473251e+08 │ 1.783876e+09 │     0.0 │ 71197419                           │ True    │ 1      │ Avenue de la Liberté              │ primary   │ drive        │ 50       │ NULL     │ 106.098 │ 2.473251e+08 │ 1.783876e+09 │ N 3    │ NULL   │ NULL   │ NULL    │ NULL   │ NULL       │ NULL    │ NULL      │ False    │
└─────────────────────────────

In [9]:
csv_size = sum(file.stat().st_size for file in csv_files if file.is_file())
parquet_size = sum(file.stat().st_size for file in parquet_files if file.is_file())
reduction_factor = csv_size/parquet_size

logger.info(f"Total file size | CSV: {humanize.naturalsize(csv_size)} | Parquet: {humanize.naturalsize(parquet_size)} | Reduction factor: {reduction_factor:2.5f}")

Streetscapes | 2025-01-30@15:36:02 | Total file size | CSV: 14.1 GB | Parquet: 3.6 GB | Reduction factor: 3.86157


 We may want to combine multiple csv files together into a single parquet file. If we use JOIN like above on the full table, we quickly run into memory issues. This is because `duckdb.sql(...)` creates an in-memory database to load the data and keep track of intermediate results. Alternatively, duckdb can create a persistent database on disk using `duckdb.connect('database_filename')`. 

In [17]:
# Perform the joins.
logger.info(f"Starting merger with '{parquet_files[0].name}'...")

# Load the first file into a table.
# We are going to use it to perform incremental joins on that table.
joined = con.read_parquet(parquet_files[0]).as_table()
for parquet_file in parquet_files[1:]:

    # Join the next Parquet file on the UUID column.
    logger.info(f"Merging '{parquet_file.name}'...")
    joined = joined.join(con.read_parquet(parquet_file).as_table(), "uuid").as_table()

# Save the final joined table to a compressed Parquet file.
logger.info("Saving merged file...")
merged_full = MERGED_DIR / "streetscapes_full.parquet"
joined.to_parquet(merged_full, compression="ZSTD")
logger.info("Done!")

Streetscapes | 2025-01-30@15:49:38 | Starting merger with 'places365.parquet'...


Streetscapes | 2025-01-30@15:49:38 | Merging 'contextual.parquet'...
Streetscapes | 2025-01-30@15:49:38 | Merging 'segmentation.parquet'...
Streetscapes | 2025-01-30@15:49:38 | Merging 'metadata_common_attributes.parquet'...
Streetscapes | 2025-01-30@15:49:38 | Merging 'perception.parquet'...
Streetscapes | 2025-01-30@15:49:38 | Merging 'ghsl.parquet'...
Streetscapes | 2025-01-30@15:49:38 | Merging 'simplemaps.parquet'...
Streetscapes | 2025-01-30@15:49:38 | Merging 'osm.parquet'...
Streetscapes | 2025-01-30@15:49:38 | Saving merged file...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Streetscapes | 2025-01-30@15:50:24 | Done!


In [18]:
# Show the merged file size
merged_size = merged_full.stat().st_size
logger.info(f"Merged file size: {humanize.naturalsize(merged_size)}")

Streetscapes | 2025-01-30@15:50:50 | Merged file size: 2.1 GB


In [19]:
con.read_parquet(merged_full).head()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┓
┃ uuid                                 ┃ source    ┃ orig_id          ┃ place                    ┃ source_right ┃ orig_id_right    ┃ glare   ┃ lighting_condition ┃ pano_status ┃ platform        ┃ quality ┃ reflection ┃ view_direction ┃ weather ┃ Bird    ┃ Ground-Animal ┃ Curb     ┃ Fence    ┃ Guard-Rail ┃ Barrier ┃ Wall     ┃ Bike-Lane ┃ Crosswalk---Plain ┃ Curb-Cut ┃ Parking ┃ Pedestrian-Area ┃ Rail-Track ┃ Road     ┃ Service-Lane ┃ Sidewalk     ┃ Bridge   ┃ Building ┃ Tunnel  ┃ Person  ┃ Bicyclist ┃ Motorcyclist ┃ Other-Rider ┃ Lane-Marking---Crosswalk ┃ Lane-Marking---General ┃ Mountain ┃ Sand    ┃ Sky      ┃ Snow    ┃ Terrain  ┃ Vegetation ┃ Water   ┃ Banner  ┃ Bench   ┃ Bike-Rack ┃ Billboard ┃ Catch-Basin ┃ CCTV-Camera ┃ Fire-Hydrant ┃ Junction-Box ┃ Mailbox ┃ Manhole ┃ Phone-Booth ┃ Pothole ┃ Street-Light ┃ Pole     ┃ Traffic-Sign-Frame ┃ Utility-Pole ┃ Traffic-Light ┃ Traffic-Sign-(Back) ┃ Traffic-Sign-(Front) ┃ Trash-Can ┃ Bicycle ┃ Boat    ┃ Bus      ┃ Car      ┃ Caravan ┃ Motorcycle ┃ On-Rails ┃ Other-Vehicle ┃ Trailer ┃ Truck   ┃ Wheeled-Slow ┃ Car-Mount ┃ Ego-Vehicle ┃ Total        ┃ green_view_index ┃ sky_view_index ┃ lat       ┃ lon        ┃ datetime_local          ┃ year  ┃ month ┃ day   ┃ hour  ┃ width   ┃ height  ┃ heading    ┃ projection_type ┃ hFoV      ┃ vFoV      ┃ sequence_index ┃ sequence_id            ┃ sequence_img_count ┃ Beautiful ┃ Boring  ┃ Depressing ┃ Lively  ┃ Safe    ┃ Wealthy ┃ urban_code ┃ urban_term          ┃ city       ┃ city_ascii ┃ city_id    ┃ city_lat ┃ city_lon ┃ country    ┃ iso2   ┃ iso3   ┃ admin_name   ┃ capital ┃ population   ┃ continent     ┃ snap_dist ┃ u            ┃ v            ┃ key     ┃ osmid      ┃ oneway  ┃ lanes  ┃ name                   ┃ highway  ┃ type_highway ┃ maxspeed ┃ junction ┃ length  ┃ from         ┃ to           ┃ ref     ┃ tunnel_1 ┃ bridge_1 ┃ service ┃ access ┃ road_width ┃ area    ┃ est_width ┃ reversed ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━

For some usecases it might be more convenient to select certain columns from different files into a single table. This can be achieved in a similar manner to the previous example. Here, we create a dictionary with the file names and columns we want to select. We also need to specify a column that is common to all files to join on. 

In [20]:
# Create dictionary choosing files and columns
selection = {
    "contextual": ['uuid', 'source', 'orig_id'],
    "osm": ['uuid', 'road_width', 'type_highway'],
    "simplemaps": ['uuid', 'city'],
    "metadata_common_attributes": ['uuid', 'lat', 'lon']
}

# Turn the selection into a list for easier traversal
selection = list(selection.items())

# Load the first file into a table.
# We are going to use it to perform incremental joins on that table.
parquet_file = PARQUET_DIR / f"{selection[0][0]}.parquet"
cols = selection[0][1]
logger.info(f"Starting merger with '{parquet_file.name}'...")
joined = con.read_parquet(parquet_file).select(*cols).as_table()

for file_name, cols in selection[1:]:

    parquet_file = PARQUET_DIR / f"{file_name}.parquet"
    logger.info(f"Merging table '{parquet_file.name}'...")

    joined = joined.join(con.read_parquet(parquet_file).select(*cols).as_table(), "uuid").as_table()

# Save the final joined table to a compressed Parquet file.
logger.info("Saving merged file...")
merged_selection = MERGED_DIR / "streetscapes_selection.parquet"
joined.to_parquet(merged_selection, compression="ZSTD")
logger.info("Done!")

Streetscapes | 2025-01-30@15:50:50 | Starting merger with 'contextual.parquet'...
Streetscapes | 2025-01-30@15:50:50 | Merging table 'osm.parquet'...
Streetscapes | 2025-01-30@15:50:50 | Merging table 'simplemaps.parquet'...
Streetscapes | 2025-01-30@15:50:50 | Merging table 'metadata_common_attributes.parquet'...
Streetscapes | 2025-01-30@15:50:50 | Saving merged file...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Streetscapes | 2025-01-30@15:50:55 | Done!


In [21]:
# Show the merged file size
merged_size = merged_selection.stat().st_size
logger.info(f"Merged file size: {humanize.naturalsize(merged_size)}")

Streetscapes | 2025-01-30@15:50:55 | Merged file size: 333.0 MB


In [22]:
# Let's inspect the new file to see if the join has worked
con.read_parquet(merged_selection).head()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ uuid                                 ┃ source    ┃ orig_id          ┃ road_width ┃ type_highway ┃ city   ┃ lat       ┃ lon       ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━┩
│ string                               │ string    │ int64            │ string     │ string       │ string │ float64   │ float64   │
├──────────────────────────────────────┼───────────┼──────────────────┼────────────┼──────────────┼────────┼───────────┼───────────┤
│ e4079908-7b3b-4580-91fa-edeb828bd9b8 │ Mapillary │ 3979559142140315 │ NULL       │ walk         │ Moscow │ 55.762861 │ 37.631832 │
│ cba79e92-79f2-4ed3-a5cc-091a7994263a │ Mapillary │ 4422290311142880 │ 2.1        │ walk         │ Moscow │ 55.756800 │ 37.638849 │
│ 43a5c070-6268-466a-9aba-37277a2f09e3 │ Mapillary │  356951482536411 │ NULL       │ drive        │ Moscow │ 55.758714 │ 37.619135 │
│ 600b12a6-bb35-4c43-ae81-16e88462fc76 │ Mapillary │  952369172240660 │ NULL       │ drive        │ Moscow │ 55.757039 │ 37.629279 │
│ 2fd0c135-6908-48ad-9d58-69a45a5378a9 │ Mapillary │ 2832119713669023 │ NULL       │ drive        │ Moscow │ 55.759773 │ 37.627337 │
└──────────────────────────────────────┴───────────┴──────────────────┴────────────┴──────────────┴────────┴───────────┴───────────┘

We are in touch with the developers of the original Open Streetscapes dataset to add these parquet files to the dataset on huggingface.